In [ ]:
import matplotlib.pyplot as plt

import planet_ruler.observation as pro

from planet_ruler.demo import make_dropdown, load_demo_parameters, display_text
from planet_ruler.plot import plot_3d_solution, plot_gradient_field_at_limb
from planet_ruler.dashboard import OutputCapture

# Choose a Target

Run the next cell and choose a target from the dropdown menu.

In [ ]:
demo = make_dropdown()
display(demo)

Load presets using the next cell and then step through the rest of the demo in order.

In [ ]:
demo_params = load_demo_parameters(demo)

# Load an Image

In [ ]:
Obs = pro.LimbObservation(
    image_filepath=demo_params["image_filepath"], fit_config=demo_params["fit_config"]
)

In [ ]:
display_text(demo_params["preamble"])

Obs.plot()

# Detect Limb

Unlike the basic limb_demo, we are going to skip limb detection! Instead we are going to make some basic assumptions about how a limb will appear visually and then let the fit take place directly on the image. Essentially, we take advantage of the fact that a horizon posesses a strong and fairly uniform brightness gradient which should be perpendicular to the planet limb at any point. We devise a loss function that tries to maximize total perpendicular gradient through the proposed limb et voila -- no detection step needed. Those of you who love physics might notice that this is essentially a flux calculation where the field is the gradient and the limb is the surface.

# Fit Planet Radius

In [ ]:
display_text(demo_params["parameter_walkthrough"])

There are a lot of fit options here between minimizers/their kwargs, multi-resolution optimization stages, smoothing settings, etc.  This is set to something robust but see if you can tweak it to get the correct answer faster!

In [ ]:
# Create capture
capture = OutputCapture(max_lines=20, line_width=70)

# Use with context manager
with capture:
    Obs.fit_limb(
        loss_function="gradient_field",
        minimizer="differential-evolution",
        resolution_stages=[4, 2],
        image_smoothing=4.0,
        kernel_smoothing=8.0,
        directional_smoothing=50,
        minimizer_preset="scipy-default",
        max_iter=3000,
        verbose=True,
        dashboard=True,
        dashboard_kwargs={
            "output_capture": capture,
            "width": 80,
            "max_warnings": 5,
            "max_hints": 4,
            "min_message_display_time": 5.0,
        },
    )

## Check Fit

### By Eye

Let's take a look at the solution and see if it makes sense. It's not challenging to end up with a fit that doesn't converge well and appears to have nothing to do with the limb in question. If that happens, don't despair! If the limb detection went okay, it usually it means some tweaking of the parameter ranges or optimizer properties is in order.

We can start by looking at the predicted limb given our optimal parameters. If any of these looks wrong, it likely is. Check your posteriors in the next section and you can try to tweak things.

In [ ]:
Obs.plot()

Because we used the gradient-field method, let's also take a look at the field at our detected limb. Arrows show the strength and direction of the gradient at a set of sampled points across our solution. A good fit means they should be strong, orthogonal to the limb, and all pointing either inward or outward.

In [ ]:
fig, ax = plot_gradient_field_at_limb(
    y_pixels=Obs.features["fitted_limb"],
    image=Obs.image,
    image_smoothing=2.0,
    directional_smoothing=50,
    directional_decay_rate=0.1,
    sample_spacing=100,
    kernel_smoothing=16.0,
)
plt.show()

We can zoom out a little to see how what the image captures compares to the rest of the planet. There is a lot to that iceberg!

Zooming out even further gives us a real view to what is going on here in three dimensions.

In [ ]:
plot_3d_solution(**Obs.best_parameters)

# The Answer!

Let's see how it did compared to the truth. We also approximate uncertainty here by calculating the covariance around the minimum.

In [ ]:
# Get radius uncertainty (automatic method selection)
print(f"Radius: {Obs.radius_km:.1f} ± {Obs.radius_uncertainty:.1f} km")

# Get uncertainty for any parameter
altitude_unc = Obs.parameter_uncertainty("h", scale_factor=1e-3, method="auto")
print(f"Altitude: {Obs.altitude_km:.1f} ± {altitude_unc['uncertainty']:.1f} km")
print(f"Method used: {altitude_unc['method']}")

pct_error = (
    (Obs.radius_km - demo_params["true_radius"]) / demo_params["true_radius"] * 100
)

print(
    f"Our estimate of {demo_params['target']}'s radius is{Obs.radius_km: .1f} \u00b1{Obs.radius_uncertainty: .0f} kilometers."
)
print(
    f"That puts us {pct_error: .1f}% away from the true value of {demo_params['true_radius']} km."
)